Let's start by loading the data.

In [1]:
import pandas as pd

employees_df = pd.read_csv("../data/employees.csv")
safehouses_df = pd.read_csv("../data/safehouses.csv")
divisions_df = pd.read_csv("../data/divisions.csv")
managers_df = pd.read_csv("../data/managers.csv")
actions_df = pd.read_csv("../data/actions.csv")


## Employees

In [2]:
employees_df.sample(5)


,EmployeeID,EmployeeName,JobTitle,Email,Phone,Manager
512,513,Christopher Mckinney,Data Scientist,christopher_mckinney@brlda.gov,\t+1-212-758-3594x7701,Mrs. Michelle James DDS
24693,24698,Andrew Fleming,Machine Learning Engineer,andrew_fleming@brlda.gov,(083)562-9918,Matthew Joseph
6804,6807,Omar Chavez,Data Analyst,omar_chavez@brlda.gov,079.884.2902,Mrs. Lynn Owens MD
12078,12081,Timothy Cohen,Business Analyst,timothy_cohen@brlda.gov,\t+1-606-026-0666x1798,Ashley Marshall
7650,7653,Austin Blair,Data Engineer,austin_blair@brlda.gov,865.692.3918,Lee Herrera DVM


In [3]:
employees_df["EmployeeName"].value_counts().head(5)


EmployeeName
Michael Smith       12
David Smith         12
Lisa Smith          11
John Smith          11
Michael Williams    10
Name: count, dtype: int64

Some duplicate employees are present in the data...

In [4]:
employees_df[employees_df["EmployeeName"] == "Michael Smith"]


,EmployeeID,EmployeeName,JobTitle,Email,Phone,Manager
4448,4451,Michael Smith,Project Manager,michael_smith@brlda.gov,001-137-479-2502x05426,Nathan Guerra
5129,5132,Michael Smith,Scrum Master,michael_smith@brlda.gov,223-274-6121x5268,David Houston
7455,7458,Michael Smith,Quality Assurance Analyst,michael_smith@brlda.gov,(202)737-6861x908,Pamela Hale
10672,10675,Michael Smith,Data Scientist,michael_smith@brlda.gov,001-192-379-3454x580,Sarah Brown
11683,11686,Michael Smith,Quality Assurance Analyst,michael_smith@brlda.gov,001-038-692-4506x839,Patrick Cruz
13957,13960,Michael Smith,Program Manager,michael_smith@brlda.gov,(471)896-2210x7208,Heather Crawford
15668,15672,Michael Smith,Scrum Master,michael_smith@brlda.gov,668.014.9966x33380,Gregory Floyd
17442,17446,Michael Smith,Data Analyst,michael_smith@brlda.gov,\t+1-544-294-4533x69316,Richard Collins
17570,17574,Michael Smith,Program Manager,michael_smith@brlda.gov,9730334254,Michele Graham
19322,19326,Michael Smith,Machine Learning Engineer,michael_smith@brlda.gov,081-057-7739,Joshua James


Are they all the same employee? The email match but the ID and Phone don't.

Let's focus on the missing data. If they removed their users, we should see some missing EmployeeID values.

In [5]:
# Check min and max EmployeeID
min_eid = employees_df["EmployeeID"].min()
max_eid = employees_df["EmployeeID"].max()
print("Min EmployeeID: ", min_eid)
print("Max EmployeeID: ", max_eid)

# Check employees ids not in the min-max range
missing_employees = set(range(min_eid, max_eid + 1)) - set(employees_df["EmployeeID"])
print("Employees ids not in the min-max range: ", missing_employees)


Min EmployeeID:  1
Max EmployeeID:  26849
Employees ids not in the min-max range:  {14976, 22602, 26188, 1423, 4284}


These might be the "ghosts" agents. Let's quickly check that all managers are also employees just in case...

In [6]:
managers = set(employees_df["Manager"])
employees = set(employees_df["EmployeeName"])

managers - employees


set()

Are these "ghosts employees" in other tables?

In [7]:
divisions_df[divisions_df["EmployeeID"].isin(missing_employees)]


,EmployeeID,EmployeeName,Division,Project,known_safehouses
1422,1423,NaN,[Division 7],[Project e-enable_holistic_models],"[14, 214, 181, 219]"
4283,4284,NaN,[Division 7],[Project repurpose_collaborative_methodologies...,"[10, 219]"
14975,14976,NaN,[Division 7],[Project transform_24/365_functionalities],"[25, 154, 231, 33, 219]"
22601,22602,NaN,[Division 7],"[Project monetize_one-to-one_mindshare, Projec...","[12, 221, 19, 18, 219]"
26187,26188,NaN,[Division 7],[Project extend_robust_action-items],"[7, 219]"


They all belong to Division 7. What actions did they take?

In [8]:
actions_df[actions_df["EmployeeID"].isin(missing_employees)].sort_values(["ActionDate"])


,EmployeeID,ActionType,ActionDate,ActionDescription,ActionLocation,ActionStatus,ActionSeverity,AssociatedProject,AssociatedDivision
41824,14976,Quantum Key Generation,1994-06-06 00:00:00,perform data mining on social media data for s...,Puerto Rico,completed,critical,Project transform_24/365_functionalities,Division 1
53036,26188,Predictive Modeling,1994-11-20 00:00:00,construct algorithms for automatic gait recogn...,Puerto Rico,failed,critical,Project extend_robust_action-items,Division 10
4283,4284,Data Clustering,1996-05-08 00:00:00,Initiate operation Networked_discrete_system_e...,Martinique,completed,high,Project repurpose_collaborative_methodologies,Division 6
81969,1423,User Profiling,1997-04-12 00:00:00,Operation Re-contextualized_attitude-oriented_...,Egypt,failed,medium,Project e-enable_holistic_models,Division 3
68673,14976,Natural Language Generation,2007-06-06 00:00:00,construct algorithms for automatic vein recogn...,Benin,completed,critical,Project transform_24/365_functionalities,Division 1
31132,4284,Quantum Resistant Cryptography,2007-09-17 00:00:00,Initiate operation Customizable_discrete_paral...,Kazakhstan,failed,critical,Project repurpose_collaborative_methodologies,Division 6
57981,4284,Machine Learning-based Intrusion Detection,2007-10-25 00:00:00,analyze communication patterns through Fully-c...,Albania,completed,high,Project repurpose_collaborative_methodologies,Division 6
28271,1423,Automated Surveillance,2009-10-18 00:00:00,Operation Down-sized_24/7_capability to develo...,Puerto Rico,failed,high,Project e-enable_holistic_models,Division 3
14975,14976,Natural Language Generation,2011-08-06 00:00:00,Initiate operation Centralized_upward-trending...,Bahrain,completed,low,Project transform_24/365_functionalities,Division 1
49450,22602,Quantum Key Generation,2012-10-22 00:00:00,Operation Digitized_methodical_structure to ap...,Ireland,failed,high,Project monetize_one-to-one_mindshare,Division 6


Some interesting things:
- They all mention the devices in their action description.
- They belong to a different Associated Division and not Division 7.

How common is for employees to do actions in other divisions? Time for OBT (one big table).

In [9]:
detailed_actions_df = actions_df.merge(
    employees_df, left_on="EmployeeID", right_on="EmployeeID", how="left"
).merge(divisions_df, left_on="EmployeeID", right_on="EmployeeID", how="left")
detailed_actions_df.sample(5)


,EmployeeID,ActionType,ActionDate,ActionDescription,ActionLocation,ActionStatus,ActionSeverity,AssociatedProject,AssociatedDivision,EmployeeName_x,JobTitle,Email,Phone,Manager,EmployeeName_y,Division,Project,known_safehouses
51345,24497,Data Clustering,2004-10-18 00:00:00,Initiate operation Decentralized_holistic_defi...,Costa Rica,completed,high,Project extend_robust_action-items,Division 1,Jonathan Day,Data Analyst,jonathan_day@brlda.gov,931.043.2180x8210,Luis Mccoy,Jonathan Day,"[Division 1, Division 2]","[Project strategize_value-added_bandwidth, Pro...",[211]
79376,25679,Natural Language Processing,2000-01-30 00:00:00,Operation Proactive_maximized_groupware to con...,Iran,failed,high,Project drive_value-added_mindshare,Division 4,Anthony Jackson,Data Scientist,anthony_jackson@brlda.gov,001-502-237-1029,Maria Vazquez DVM,Anthony Jackson,"[Division 4, Division 6, Division 4]","[Project unleash_front-end_models, Project dri...","[35, 48]"
34640,7792,Image Analysis,1997-09-01 00:00:00,perform sentiment analysis on customer reviews...,Cambodia,failed,low,Project deliver_visionary_web-readiness,Division 9,Bradley Davis,Statistician,bradley_davis@brlda.gov,3517150454,Aaron Todd,Bradley Davis,[Division 9],"[Project deliver_visionary_web-readiness, Proj...","[45, 55]"
82486,1940,Quantum Computing Decoding,2018-08-25 00:00:00,Operation Quality-focused_eco-centric_time-fra...,Israel,failed,critical,Project mesh_cutting-edge_experiences,Division 7,John Thompson,Business Analyst,john_thompson@brlda.gov,\t+1-462-677-1117x520,Angela Carlson,John Thompson,"[Division 8, Division 7]",[Project mesh_cutting-edge_experiences],"[133, 193]"
36370,9522,DDoS Attack Detection,2007-03-23 00:00:00,Operation Public-key_systematic_methodology to...,Sierra Leone,completed,medium,Project scale_back-end_interfaces,Division 9,Justin Young,Data Scientist,justin_young@brlda.gov,858.613.0262,Thomas Stewart,Justin Young,"[Division 9, Division 2, Division 5]",[Project scale_back-end_interfaces],"[67, 2, 49, 142]"


In [10]:
def check_division_associated_division(row):
    return row["AssociatedDivision"] in row["Division"]


detailed_actions_df[
    ~detailed_actions_df.apply(check_division_associated_division, axis=1)
].sample(5)


,EmployeeID,ActionType,ActionDate,ActionDescription,ActionLocation,ActionStatus,ActionSeverity,AssociatedProject,AssociatedDivision,EmployeeName_x,JobTitle,Email,Phone,Manager,EmployeeName_y,Division,Project,known_safehouses
92138,11592,Data Anonymization,2006-11-18 00:00:00,analyze network logs for detecting command and...,Andorra,completed,critical,Project mesh_cutting-edge_experiences,Division 7,Gabriel Smith,Data Scientist,gabriel_smith@brlda.gov,596.995.7536x22790,Debra Gonzalez,Gabriel Smith,"[Division 1, Division 3]",[Project mesh_cutting-edge_experiences],"[219, 5, 133]"
40147,13299,Customer Support Interaction Sentiment Analysis,2015-05-19 00:00:00,Operation Cloned_didactic_implementation to ex...,Benin,failed,low,Project embrace_magnetic_systems,Division 7,Dillon Moran,Quality Assurance Analyst,dillon_moran@brlda.gov,001-124-430-5696x9265,Nicholas Davila,Dillon Moran,"[Division 3, Division 2, Division 9]","[Project embrace_magnetic_systems, Project e-e...","[140, 4, 1]"
87019,6473,Social Media Post Analysis,2017-12-25 00:00:00,establish covert presence through Optimized_st...,Slovenia,failed,medium,Project embrace_transparent_networks,Division 7,Melissa Thompson,Machine Learning Engineer,melissa_thompson@brlda.gov,001-896-224-3290,David Rogers,Melissa Thompson,"[Division 3, Division 5, Division 3]",[Project embrace_transparent_networks],"[157, 231, 51, 232]"
60427,6730,Malicious Code Detection,1999-06-11 00:00:00,Initiate operation Optional_multimedia_paralle...,Venezuela,completed,high,Project facilitate_mission-critical_ROI,Division 7,Michael Duncan,Machine Learning Engineer,michael_duncan@brlda.gov,\t+1-112-045-6111x9914,Alexander Mccullough,Michael Duncan,"[Division 8, Division 10, Division 6]",[Project facilitate_mission-critical_ROI],"[188, 19, 141]"
75981,22284,Covert Sentiment-based Social Engineering,2018-06-19 00:00:00,perform sentiment analysis on online forums th...,Wallis and Futuna,failed,critical,Project synergize_B2C_technologies,Division 7,Elizabeth Garcia,Business Analyst,elizabeth_garcia@brlda.gov,234.106.8187x788,Paul Robertson,Elizabeth Garcia,"[Division 9, Division 5, Division 4]",[Project synergize_B2C_technologies],"[220, 151]"


Some people did actions associated to Division 7 but is not reflected in the Employee table Divisions.

In [11]:
detailed_actions_df[
    ~detailed_actions_df.apply(check_division_associated_division, axis=1)
]["AssociatedDivision"].value_counts()


AssociatedDivision
Division 7     2284
Division 6        7
Division 3        4
Division 1        4
Division 10       3
Name: count, dtype: int64

Now, most of the people that did actions in Division 7 are missclassified, but there are some that did actions in the divisions 1, 3, 6, and 10. These are our suspects.

## Safehouses

In [12]:
safehouses_df.sample(5)


,ID,City,Address,Latitude,Longitude
198,1,New York,"87 Beekman Avenue, Vernon Park, City of Mount ...",40.908226,-73.823729
186,6,London,"20 Robson Avenue, London, NW10 3SZ, United Kin...",51.543403,-0.236712
34,183,Lima,"Avenida Coronel Néstor Gambetta, 200 Millas, C...",-12.004199,-77.124531
81,99,Santa Cruz de la Sierra,"RN9: La Guardia-Abapó, Municipio La Guardia, B...",-17.902468,-63.252284
187,6,Paris,"53 Rue Victor Hugo, 95200 Sarcelles, France",49.001127,2.385609


In [13]:
# Map of safehouses with Latitude and Longitude
import folium

safehouses_map = folium.Map(
    location=[safehouses_df["Latitude"].mean(), safehouses_df["Longitude"].mean()],
    zoom_start=4,
)

for index, row in safehouses_df.iterrows():
    folium.Marker([row["Latitude"], row["Longitude"]], popup=row["ID"]).add_to(
        safehouses_map
    )

safehouses_map


Cute, but not very useful for now, let's explore divisions.

## Divisions

In [14]:
divisions_df


,EmployeeID,EmployeeName,Division,Project,known_safehouses
0,1,Kelly Rios,[Division 1],[Project deliver_visionary_web-readiness],"[232, 1, 73, 217]"
1,2,Madison Barr,"[Division 6, Division 3]",[Project repurpose_collaborative_methodologies],"[192, 26, 118, 4]"
2,3,Sue Anderson,"[Division 5, Division 1, Division 10]",[Project repurpose_collaborative_methodologies],"[19, 8, 130, 50]"
3,4,Laura Carlson,"[Division 9, Division 9, Division 2]",[Project streamline_proactive_e-markets],"[15, 232]"
4,5,Carrie Ali,"[Division 3, Division 6]",[Project deliver_visionary_web-readiness],"[158, 118]"
...,...,...,...,...,...
26844,26845,Christopher Riley,"[Division 8, Division 10, Division 5]","[Project drive_value-added_mindshare, Project ...",[226]
26845,26846,Eric Chan,"[Division 2, Division 6, Division 1]",[Project deliver_visionary_web-readiness],"[15, 99, 58]"
26846,26847,Amy Vazquez,"[Division 8, Division 10, Division 5]","[Project embrace_transparent_networks, Project...",[22]
26847,26848,Clifford Reyes,"[Division 9, Division 2, Division 6]",[Project disintermediate_distributed_experienc...,[98]


In [15]:
# TODO: Explode Division and Project


In [16]:
# TODO: Some divisions are repeated in Division


### Managers

In [17]:
managers_df.sample(5)


,ManagerName,Employee_1,Employee_2,Employee_3,Employee_4,Employee_5,Employee_6,Employee_7,Employee_8,Employee_9,...,Employee_20,Employee_21,Employee_22,Employee_23,Employee_24,Employee_25,Employee_26,Employee_27,Employee_28,Employee_29
3489,Nancy Fleming,Paul Shannon,Adrian Lopez,Nicholas Taylor,David Mcbride,Mr. Jerry Williams II,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,Teresa Johnson,Nathan Gibson,Thomas Jones MD,Adam Stark,Martha Sanchez,Monique Campbell,Carla Hall,William Harrington,Anthony Paul,Luke Mills,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2060,David Hubbard,Mary Miller,Denise Williams,Michael Grant,Alisha Turner,Amy Soto,Krista Walker,Jasmine Ramirez,Jennifer Mills,Steve Krause,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3126,Joshua Allen,Jenna Shaffer,Christopher Stewart,Heather Ochoa,Briana Sanchez,Katherine Adams,Katherine Kirk,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,Nicole Williams,Lucas Campos,David James,Tracy Franklin,Diana Zimmerman,Michelle Anderson,Brett Johnson,Cassandra Gallegos,Tanner Thompson,Jeremy Hopkins,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Transform to long format
clean_managers_df = (
    managers_df.melt(
        id_vars=["ManagerName"], value_name="EmployeeName", var_name="EmployeeNumber"
    )
    .dropna()
    .drop(columns=["EmployeeNumber"])
)

clean_managers_df.sample(5)


,ManagerName,EmployeeName
28030,Jeffery Jones,Michael Stevens
6700,Alexandra Gibson,Erica Duran
1289,Sara Wilcox,Kimberly Webb
41082,Brittany Olson,Kimberly Rollins
6816,Monica Evans,Patricia Rollins


In [19]:
managers_df[managers_df["Employee_1"].isna()]


,ManagerName,Employee_1,Employee_2,Employee_3,Employee_4,Employee_5,Employee_6,Employee_7,Employee_8,Employee_9,...,Employee_20,Employee_21,Employee_22,Employee_23,Employee_24,Employee_25,Employee_26,Employee_27,Employee_28,Employee_29
3686,Jessica Stone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Jessica Stone is the only manager without any employees (unless there are NaNs gaps).

In [20]:
clean_managers_df[clean_managers_df["EmployeeName"] == "Jessica Stone"]


,ManagerName,EmployeeName
17210,Christopher Mckenzie,Jessica Stone


In [21]:
employees_df[employees_df["EmployeeName"] == "Jessica Stone"]


,EmployeeID,EmployeeName,JobTitle,Email,Phone,Manager
11560,11563,Jessica Stone,Data Scientist,jessica_stone@brlda.gov,001-234-563-9331,Christopher Mckenzie


In [22]:
divisions_df[divisions_df["EmployeeName"] == "Jessica Stone"]


,EmployeeID,EmployeeName,Division,Project,known_safehouses
11562,11563,Jessica Stone,[Division 4],"[Project mesh_cutting-edge_experiences, Projec...","[18, 42, 44]"


In [23]:
actions_df[actions_df["EmployeeID"] == 11563]


,EmployeeID,ActionType,ActionDate,ActionDescription,ActionLocation,ActionStatus,ActionSeverity,AssociatedProject,AssociatedDivision
11562,11563,Gesture Recognition,2016-06-21 00:00:00,build systems for automatic object detection i...,Syrian Arab Republic,failed,critical,Project unleash_front-end_models,Division 4
38411,11563,Automated Social Media Profiling,2012-08-31 00:00:00,Operation Optimized_real-time_artificial_intel...,Botswana,failed,low,Project unleash_front-end_models,Division 4
65260,11563,Object Recognition,1996-05-13 00:00:00,perform data mining on financial transactions ...,Congo,completed,medium,Project unleash_front-end_models,Division 4
92109,11563,Covert Facial Recognition,1999-05-26 00:00:00,Initiate operation Visionary_coherent_architec...,Italy,completed,critical,Project unleash_front-end_models,Division 4


# Random

In [24]:
data = pd.merge(employees_df, divisions_df, on=["EmployeeID", "EmployeeName"])
data = pd.merge(data, clean_managers_df, on="EmployeeName", how="left")
data = data[data["Manager"] == data["ManagerName"]]
data.shape


(28316, 10)

In [25]:
jessica = data[data["EmployeeName"] == "Jessica Stone"]


In [26]:
MANAGER_WEIGHT = 100
DIVISION_WEIGHT = 10
PROJECT_WEIGHT = 1


def calculate_similarity(emp1, emp2):
    score = 0
    if emp1["ManagerName"] == emp2["ManagerName"]:
        score += MANAGER_WEIGHT
    if bool(set(emp1["Division"]) & set(emp2["Division"])):
        score += DIVISION_WEIGHT
    if bool(set(emp1["Project"]) & set(emp2["Project"])):
        score += PROJECT_WEIGHT
    return score


from itertools import combinations


def top_similarities(data):
    # Create a list to store the results
    similarities = []

    # Calculate the similarity score for each pair of employees
    # for emp1, emp2 in combinations(data.iterrows(), 2):
    #     score = calculate_similarity(emp1[1], emp2[1])
    #     similarities.append(((emp1[1]["EmployeeName"], emp2[1]["EmployeeName"]), score))
    # Calculate the similarity score for each pair of employees
    for emp2 in data.iterrows():
        score = calculate_similarity(jessica.iloc[0], emp2[1])
        similarities.append([emp2[1]["EmployeeName"], score])

    # Sort the results by score in descending order and take the top 100
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_similarities = similarities[:200]

    # Convert the results to a DataFrame
    top_similarities_df = pd.DataFrame(
        top_similarities, columns=["Employee Pair", "Similarity Score"]
    )

    return top_similarities_df


top_similarities_df = top_similarities(data)
top_similarities_df.head(20)


,Employee Pair,Similarity Score
0,Kent Hernandez,111
1,Daniel Washington,111
2,Robert Walter,111
3,Jenna Ray,111
4,Jessica Stone,111
5,Tiffany Oconnell,111
6,Brittany Murphy,111
7,Michael Freeman,111
8,Tyler Owens,111
9,Daniel Daniel,111
